In [2]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)
from combiners import EpochsCombiner
from typing import *
import mne
import tensorflow as tf
import mneflow as mf
import matplotlib.pyplot as plt
import numpy as np
from utils.data_management import dict2str
from lfcnn_vis import plot_patterns
from LFCNN_decoder import SpatialParameters, TemporalParameters, ComponentsOrder, Predictions
import pickle
from utils.machine_learning import one_hot_decoder
import sklearn.metrics as sm
from utils.machine_learning.confusion import ConfusionEstimator

def read_pkl(path: str) -> Any:
    with open(
            path,
            'rb'
        ) as file:
        content = pickle.load(
            file
        )
    return content

2022-02-21 17:16:01.556260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-21 17:16:01.556283: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:


subjects_path = '../Source/Subjects/'

for subject_name in os.listdir(subjects_path):
    
    if subject_name == 'Pse_Udo':
        continue
    
    subject_path = os.path.join(subjects_path, subject_name)

    predictions = read_pkl(
        os.path.join(subject_path, 'Predictions', 'LM&LI_vs_RM&RI_B1-B8_pred.pkl')
    )

    y_t = one_hot_decoder(predictions.y_true)

    y_p = one_hot_decoder(predictions.y_p)

    print(subject_name)
    tn, fp, fn, tp = sm.confusion_matrix(y_t, y_p).ravel()
    ce = ConfusionEstimator(tp, tn, fp, fn)

    print(
        f'\tT\tF\n'\
        f'P\t{tp}\t{fp}\n'\
        f'N\t{tn}\t{fn}\n'
        f'Accuracy: {ce.acc}\n'\
        f'Specificity: {ce.spec}\n'\
        f'Sensitivity: {ce.sens}\n'
    )

Ga_Fed_06
	T	F
P	32	35
N	33	36
Accuracy: 0.47794117647058826
Specificity: 0.4852941176470588
Sensitivity: 0.47058823529411764

Fe_To_08
	T	F
P	34	35
N	33	33
Accuracy: 0.4962962962962963
Specificity: 0.4852941176470588
Sensitivity: 0.5074626865671642

Az_Mar_05
	T	F
P	32	29
N	42	31
Accuracy: 0.5522388059701493
Specificity: 0.5915492957746479
Sensitivity: 0.5079365079365079

Ku_EL_09
	T	F
P	25	32
N	46	33
Accuracy: 0.5220588235294118
Specificity: 0.5897435897435898
Sensitivity: 0.43103448275862066

Pr_An_04
	T	F
P	37	32
N	34	33
Accuracy: 0.5220588235294118
Specificity: 0.5151515151515151
Sensitivity: 0.5285714285714286

Ru_Ek_07
	T	F
P	36	32
N	33	33
Accuracy: 0.5149253731343284
Specificity: 0.5076923076923077
Sensitivity: 0.5217391304347826

Se_Tu_03
	T	F
P	46	29
N	33	29
Accuracy: 0.5766423357664233
Specificity: 0.532258064516129
Sensitivity: 0.6133333333333333

Te_Ali_11
	T	F
P	40	30
N	40	26
Accuracy: 0.5882352941176471
Specificity: 0.5714285714285714
Sensitivity: 0.6060606060606061



In [9]:



# for true, prediction in zip(y_t, y_p):
#     print(true, prediction)

0.47794117647058826